In [98]:
import selenium
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from time import sleep

import requests 
from bs4 import BeautifulSoup


# Заходим на сайт "Такие дела" - будем искать новости по слову "коронавирус", листая страницы с помощью Selenium
# Наша цель: получить dataframe с 7 столбцами (и >200 строками)
# Алгоритм действий: ищем иконку с лупой, нажимаем на нее
# После нажатия появляется строка с инпутом
# Нажимаем на строку для установления курсора и печатаем "коронавирус"
# Нажимаем enter, попадаем на страницу с новостями про "коронавирус"
# Новости можно листать дальше, нажимая на кнопку "Далее". Будем это делать в цикле for
# Во время прочтения каждой страницы будем применять функцию, которая будет заходить в каждую новость и забирать информацию

https://takiedela.ru/

In [99]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://takiedela.ru/') # аналог страницы браузера Chrome

# находим иконку лупы поиска - правый верхний угол вкладки сайта
xpath_query = '/html/body/header/nav[1]/button[2]'
search = driver.find_element_by_xpath(xpath_query)
# кликаем на нее, чтобы появился инпут для поиска
search.click()



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [C:\Users\Polina Lenovo\.wdm\drivers\chromedriver\win32\96.0.4664.45\chromedriver.exe] found in cache
<ipython-input-99-f4f62df9b0d3>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())
<ipython-input-99-f4f62df9b0d3>:6: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search = driver.find_element_by_xpath(xpath_query)


In [100]:
# находим инпут поиска, чтобы установить курсор
xpath_query_input = '/html/body/header/form/div/input'
search = driver.find_element_by_xpath(xpath_query_input)
# кликаем в него
search.click()

<ipython-input-100-239b5cbb1e22>:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search = driver.find_element_by_xpath(xpath_query_input)


In [101]:
# будем искать новости про коронавирус
search_ = "коронавирус"
# печатаем фразу в окне для поиска
search.send_keys(search_)
# нажимаем кнопку enter
search.send_keys(Keys.RETURN)
sleep(3)

In [102]:
# оказываемся на странице с новостями 
# принаем политику кук, которая выскакивает снизу, нажимая кнопку "Понятно"
xpath_ = '/html/body/div[1]/button'
search_ = driver.find_element_by_xpath(xpath_)
# кликаем на кнопку "Понятно"
search_.click()

<ipython-input-102-f9df1cf47fd9>:4: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_ = driver.find_element_by_xpath(xpath_)


In [103]:
# инициализируем структуру данных о новостях, которые мы будем собирать
# news_date - дата публикации
# news_name - это заголовок новости
# news_descr - это краткое описание новости
# news_link - ссылка на конкретную новость
# news_publication_time - время публикации
# news_fulltext - полный текст новости
# news_tags - тэги новости

data = {'news_date': [ ],
        'news_name': [ ],
        'news_descr': [ ],
        'news_link': [ ],
        'news_publication_time': [ ],
        'news_fulltext': [ ],
        'news_tags': [ ]}

In [104]:
# функция, которая позволяет по url страницы с новостями заходить в каждую отд. новость и забирать данные

def parcing_one_page_of_takie_dela(url):
    
    response_ = requests.get(url)
    page_tree = BeautifulSoup(response_.content, 'html.parser')
    
    a = page_tree.find_all('dd',{'class':'b-found__result__cont'}) # здесь - все news_name, news_descr, news_link страницы
    b = page_tree.find_all('dt',{'class':'b-found__result__date'}) # здесь - все news_date страницы
    
    # на каждой странице новостей - 15 штук, поэтому обработаем их циклом
    # соберем информацию о каждой конкретной новости
    
    for i in range(15):
        
        link = a[i].a.get('href')
        
        # !из всех новостей было четыре, страницы которых по вёрстке полностью отличаются от других
        # пропускаем их для упрощения
        # можно пройти по их ссылкам, чтобы посмотреть отличие страниц
            
        if(link == 'https://takiedela.ru/2021/11/boleyut-tyazhelo/'):
            continue
        elif(link == 'https://takiedela.ru/2021/06/privitsya-i-zabolet/'):
            continue
        elif(link == 'https://takiedela.ru/2021/02/koronavirus-vnutrennyaya-mikrovoyna/'):
            continue 
        elif(link == 'https://nuzhnapomosh.ru/funds/dom-s-mayakom/'):
            continue # эта написана в сотрудничестве с фондом Нужна помощь
            
        
        data['news_link'].append(a[i].a.get('href'))
        
        data['news_date'].append(b[i].text)
        
        name = a[i].text.strip()
        name = name.replace('\n',' ') # убираем мусор из текста
        name = name.replace('\xa0',' ') # убираем мусор из текста
        data['news_name'].append(name)
        
        descr = a[i].p.text.strip()
        descr = descr.replace('\n',' ') # убираем мусор из текста
        descr = descr.replace('\xa0',' ') # убираем мусор из текста
        descr = descr.replace('\xa015',' ') # убираем мусор из текста
        data['news_descr'].append(descr)
        
        
        response__ = requests.get(link) # страница конкретной новости 
        particular_news_tree = BeautifulSoup(response__.content, 'html.parser')
        
        
        # вырезаем время публикации из строки date+time с помощью среза строки [-5:]
        c = particular_news_tree.find('time',{'class':'b-post__head-info__block'})
        
        if(c):
            data['news_publication_time'].append(c.text.strip()[-5:])
        else:
            data['news_publication_time'].append('00:00')
            
        # полный текст статья разбит на параллельные параграфы p 
        # параграфов может быть несколько, поэтому собираем их в цикле
        d = particular_news_tree.find('div',{'class':'b-post__content'}).find_all('p')
        text = ""
        for j in d:
            text = text + j.text.strip()
        text = text.replace('\xa0',' ') # убираем мусор из текста
        
        data['news_fulltext'].append(text)
        
        # у статьи может быть >=1 тегов, поэтому собираем их в цикле
        e = particular_news_tree.find_all('li',{'class':'b-post__tags__item'})
        tags = [ ]
        for k in e:
            tags.append(k.a.text)
        data['news_tags'].append(tags)
        

    #return data    

In [105]:
parcing_one_page_of_takie_dela(driver.current_url) # применим функцию к самой первой странице
# вот этой: https://takiedela.ru/page/1/?s=%D0%BA%D0%BE%D1%80%D0%BE%D0%BD%D0%B0%D0%B2%D0%B8%D1%80%D1%83%D1%81

In [106]:
data

{'news_date': ['24. 11. 2021',
  '14. 11. 2021',
  '11. 11. 2021',
  '09. 11. 2021',
  '08. 11. 2021',
  '28. 10. 2021',
  '27. 10. 2021',
  '24. 10. 2021',
  '21. 10. 2021',
  '13. 10. 2021',
  '13. 10. 2021',
  '12. 10. 2021',
  '05. 10. 2021',
  '04. 10. 2021'],
 'news_name': ['Голикова анонсировала регистрацию вакцины от коронавируса для подростков «Спутник М»                                                                       Новая вакцина может поступить в гражданский оборот в конце декабря',
  'В ЕАО трое сотрудников скорой помощи, которые хотели уволиться из-за нежелания делать прививку, заразились коронавирусом                                                                       Медработники объясняли свое решение тем, что постоянно работают в госпитале с пациентами с коронавирусом и никто еще не заболел',
  '«Скоро „вакцина“ сработает и вы умрете». После кибератаки бот «Госуслуг» рассылал сообщения, отрицающие коронавирус                                                    

In [107]:
# теперь мы находимся на первой странице с новостями про коронавирус
# вот тут: https://takiedela.ru/page/1/?s=%D0%BA%D0%BE%D1%80%D0%BE%D0%BD%D0%B0%D0%B2%D0%B8%D1%80%D1%83%D1%81
# пробуем в первый раз нажать на "Далее" внизу страницы, чтобы перейти на вторую страницу новостей

In [108]:
# находим кнопку "Далее"
xpath = '//*[@id="wrapper"]/div[1]/div/div/nav/div/a'
search = driver.find_element_by_xpath(xpath)
# кликаем на неё
search.click()

<ipython-input-108-3ba6c21fb543>:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search = driver.find_element_by_xpath(xpath)


In [109]:
# теперь мы находимся на второй странице ленты новостей про коронавирус
# вот тут: https://takiedela.ru/page/2/?s=%D0%BA%D0%BE%D1%80%D0%BE%D0%BD%D0%B0%D0%B2%D0%B8%D1%80%D1%83%D1%81

In [110]:
# переходим в цикле на еще несколько страниц, при этом применяя функцию парсинга
# на каждой стр. 15 новостей -> пройдемся по 15 страницам

for i in range(0, 15):
    
    # собираем информацию о новостях на текущей странице
    parcing_one_page_of_takie_dela(driver.current_url)
    
    # находим кнопку "Далее"
    xpath = '//*[@id="wrapper"]/div[1]/div/div/nav/div/a[2]'
    sleep(2)
    search = driver.find_element_by_xpath(xpath)
    sleep(5)
    # кликаем на нее
    search.click()
    sleep(3)

<ipython-input-110-001d6c84eb0c>:10: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search = driver.find_element_by_xpath(xpath)


In [112]:
len(data['news_date']) # всего д.б. получиться 240 новостей (15*16), но мы исключили 4 новости в функции парсинга

236

In [114]:
import pandas as pd

df = pd.DataFrame(data)
df

,news_date,news_name,news_descr,news_link,news_publication_time,news_fulltext,news_tags
0,24. 11. 2021,Голикова анонсировала регистрацию вакцины от к...,Новая вакцина может поступить в гражданский об...,https://takiedela.ru/news/2021/11/24/vakcina-s...,15:59,Вице-премьер Татьяна Голикова анонсировала рег...,"[Вакцина, коронавирус, подростки]"
1,14. 11. 2021,"В ЕАО трое сотрудников скорой помощи, которые ...","Медработники объясняли свое решение тем, что п...",https://takiedela.ru/news/2021/11/14/mediki-za...,12:56,Трое сотрудников скорой помощи в городе Облучь...,"[вакцинация, врачи, коронавирус]"
2,11. 11. 2021,«Скоро „вакцина“ сработает и вы умрете». После...,"В Минцифры заявили, что это самая мощная атака...",https://takiedela.ru/news/2021/11/11/bot-gosus...,21:27,После мощной атаки на сайт «Госуслуги» бот пор...,"[интернет, коронавирус, россия]"
3,09. 11. 2021,В Санкт-Петербурге ввели обязательную вакцинац...,Первым компонентом вакцины граждане должны при...,https://takiedela.ru/news/2021/11/09/obyazatel...,18:10,В Санкт-Петербурге введена обязательная вакцин...,"[вакцинация, коронавирус, пожилые]"
4,08. 11. 2021,"В России ввели новый сертификат, подтверждающи...",В оперштабе назвали это сбоем системы «Госуслуг»,https://takiedela.ru/news/2021/11/08/novyy-ser...,09:23,В России с 8 ноября введены сертификаты о вакц...,"[вакцинация, коронавирус, пандемия]"
...,...,...,...,...,...,...,...
231,16. 07. 2020,«Агора»: ограничения из-за коронавируса привел...,Технологии слежки в связи с коронавирусом испо...,https://takiedela.ru/news/2020/07/16/doklad-sl...,11:11,В России введение ограничительных мер из-за ко...,[коронавирус]
232,15. 07. 2020,Власти Чечни закрыли границы для жителей други...,«Открытие курортного сезона для туристов и гос...,https://takiedela.ru/news/2020/07/15/vlasti-ch...,17:01,В Чеченской республике временно запретили въез...,[коронавирус]
233,15. 07. 2020,Все жители Москвы с 16 июля смогут бесплатно с...,Для сдачи анализов на коронавирус будут открыт...,https://takiedela.ru/news/2020/07/15/besplatny...,13:15,Все москвичи смогут бесплатно сдать ПЦР-тест н...,[коронавирус]
234,12. 07. 2020,В России завершились испытания вакцины от коро...,Исследование началось 18 июня,https://takiedela.ru/news/2020/07/12/vakcina-o...,10:42,В Москве Сеченовский университет завершил испы...,"[коронавирус, медицинская помощь]"
